In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import imageio
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] ="0"

In [3]:
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

In [4]:
## WITH 20 options lorl states and horizon 10

# BASE_PATH="/atlas/u/divgarg/projects/Language-RL/hrl/outputs/2021-12-18/21-35-16/checkpoints/LorlEnv-v0-40108-traj_option-2021-12-18-21:35:16/"
# ITER = 360

In [5]:
## WITH 20 OPTIONS LORL IMAGES, HORIZON 10

# BASE_PATH = "/atlas/u/divgarg/projects/Language-RL/hrl/outputs/2021-12-22/11-59-46/checkpoints/LorlEnv-v0-40108-traj_option-2021-12-22-11:59:46"
# ITER = 500

In [6]:
## WITH 20 OPTIONS LORL IMAGES, HORIZON 10

#BASE_PATH = "/atlas/u/divgarg/projects/Language-RL/hrl/outputs/2022-01-07/14-24-30/checkpoints/LorlEnv-v0-40108-traj_option-2022-01-07-14:24:31"
#ITER = 70 # 20, 70
# eval_episode_factor = 10

In [7]:
#BASE_PATH = "/atlas/u/divgarg/projects/Language-RL/hrl/outputs/2022-01-13/11-21-27/checkpoints/LorlEnv-v0-40108-traj_option-2022-01-13-11:21:28"
BASE_PATH = "/atlas/u/divgarg/projects/Language-RL/hrl/outputs/2022-01-13/11-25-35/checkpoints/LorlEnv-v0-40108-vanilla-2022-01-13-11:25:35"
ITER = 500

In [8]:
CKPT=f"{BASE_PATH}/model_{ITER}.ckpt"

In [9]:
with initialize("conf"):
    cfg = compose(config_name="config.yaml", overrides=["eval=True", "method=option_dt", f"checkpoint_path={CKPT}"])
    print(cfg)

{'cuda_deterministic': False, 'wandb': True, 'seed': 0, 'resume': False, 'checkpoint_path': '/atlas/u/divgarg/projects/Language-RL/hrl/outputs/2022-01-13/11-25-35/checkpoints/LorlEnv-v0-40108-vanilla-2022-01-13-11:25:35/model_500.ckpt', 'eval': True, 'render': False, 'render_path': './eval_${env.name}/', 'batch_size': 512, 'max_iters': 500, 'warmup_steps': 2500, 'lr_decay': 0.1, 'decay_steps': 100000, 'option_dim': 128, 'codebook_dim': 16, 'parallel': True, 'savedir': 'checkpoints', 'savepath': None, 'method': 'option_dt', 'use_iq': False, 'learning_rate': 1e-05, 'lm_learning_rate': 1e-07, 'weight_decay': 0.0001, 'os_learning_rate': 1e-05, 'trainer': {'device': None, 'state_il': False, 'num_eval_episodes': 5, 'eval_every': 5, 'K': '${model.K}'}, 'model': {'name': 'traj_option', 'horizon': 10, 'K': 10, 'train_lm': True, 'use_iq': '${use_iq}', 'method': '${model.name}', 'state_reconstruct': False, 'lang_reconstruct': False}, 'env': {'skip_words': ['go', 'to', 'the', 'a', '[SEP]'], 'name'

In [10]:
from main import *
import ast

def evaluate(cfg):
    # load saved arguments
    checkpoint = torch.load(cfg.checkpoint_path)
    args = checkpoint['config']
    max_length = checkpoint['train_dataset_max_length']
    args.eval = cfg.eval
    args.render = cfg.render
    args.checkpoint_path = cfg.checkpoint_path
    device = cfg.trainer.device

    # args.env.eval_episode_factor = 1
    # Set num train_trajs to something small
    args.train_dataset.num_trajectories = 1000
    print(OmegaConf.to_yaml(args))

    args.method = args.model.name

    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    num_eval_episodes = args.trainer.num_eval_episodes

    if not args.env.eval_offline:
        env = gym.make(args.env.name)
        #env.seed(args.seed)
        state_dim = args.env.state_dim
        if isinstance(state_dim, str):
            state_dim = ast.literal_eval(state_dim)
        action_dim = args.env.action_dim

    if 'BabyAI' in args.env.name:
        state_dim += 4*args.env.use_direction

    train_dataset_args = dict(args.train_dataset)
    batch_size = args.batch_size

    if 'BabyAI' in args.env.name:
        train_dataset = ExpertDataset(**train_dataset_args, use_direction=args.env.use_direction)
    elif 'Lorl' in args.env.name:
        # train_dataset_args also contains a split here for the validation data size
        train_dataset = ExpertDataset(**train_dataset_args, use_state=args.env.use_state)
    else:
        raise NotImplementedError
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                              shuffle=True, drop_last=True)
    del train_dataset
    
    eval_episode_factor =  args.env.eval_episode_factor 
    
    if args.method == 'traj_option':
        args.option_selector.option_transformer.max_length = int(max_length)
        args.option_selector.option_transformer.max_ep_len = eval_episode_factor * int(max_length)
        # args.option_selector.option_transformer.output_attention = True

    option_selector_args = dict(args.option_selector)
    option_selector_args['state_dim'] = state_dim
    option_selector_args['option_dim'] = args.option_dim
    option_selector_args['codebook_dim'] = args.codebook_dim
    # option_selector_args['option_transformer']['output_attention'] = True
    
    state_reconstructor_args = dict(args.state_reconstructor)
    lang_reconstructor_args = dict(args.lang_reconstructor)
    decision_transformer_args = {'state_dim': state_dim,
                                 'action_dim': action_dim,
                                 'option_dim': args.option_dim,
                                 'discrete': args.env.discrete,
                                 'hidden_size': args.dt.hidden_size,
                                 'use_language': args.method == 'vanilla',
                                 'use_options': args.method != 'vanilla',
                                 'max_length': max_length if args.method != 'traj_option' else args.model.K,
                                 # setting this to be sufficiently large so that there is enough of a buffer during eval
                                 'max_ep_len': eval_episode_factor*max_length,
                                 'action_tanh': False,
                                 'n_layer': args.dt.n_layer,
                                 'n_head': args.dt.n_head,
                                 'n_inner': 4*args.dt.hidden_size,
                                 'activation_function': args.dt.activation_function,
                                 'n_positions': args.dt.n_positions,
                                 'resid_pdrop': args.dt.dropout,
                                 'attn_pdrop': args.dt.dropout,
                                 }
    hrl_model_args = dict(args.model)
    iq_args = cfg.iq

    model = HRLModel(option_selector_args, state_reconstructor_args,
                     lang_reconstructor_args, decision_transformer_args, iq_args, device, **hrl_model_args)

    model = model.to(device=device)

    trainer_args = dict(args.trainer)

    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        optimizer=None,
        train_loader=train_loader,
        env=env,
        val_loader=None,
        scheduler=None,
        **trainer_args
    )

    # Restore trainer from checkpoint
    ## TEMP: DISABLE LOADING CHECKPOINT
    trainer.load(args.checkpoint_path)
    return model, tokenizer, train_loader, env, args

In [11]:
model, tokenizer, train_loader, env, args = evaluate(cfg)
#max_length = args.option_selector.option_transformer.max_length

cuda_deterministic: false
wandb: true
seed: 0
resume: false
checkpoint_path: /atlas/u/divgarg/projects/Language-RL/hrl/outputs/2022-01-13/11-25-35/checkpoints/LorlEnv-v0-40108-vanilla-2022-01-13-11:25:35/model_500.ckpt
eval: true
render: false
render_path: ./eval_${env.name}/
batch_size: 512
max_iters: 500
warmup_steps: 2500
lr_decay: 0.1
decay_steps: 100000
option_dim: 128
codebook_dim: 16
parallel: true
savedir: checkpoints
savepath: /atlas/u/divgarg/projects/Language-RL/hrl/outputs/2022-01-13/11-25-35/checkpoints/LorlEnv-v0-40108-vanilla-2022-01-13-11:25:35
method: vanilla
use_iq: false
learning_rate: 1.0e-05
lm_learning_rate: 1.0e-07
weight_decay: 0.0001
os_learning_rate: 1.0e-05
trainer:
  device: cuda:0
  state_il: false
  num_eval_episodes: 5
  eval_every: 5
  K: ${model.K}
model:
  name: vanilla
  horizon: 5
  K: 5
  train_lm: true
  method: ${model.name}
  state_reconstruct: false
  lang_reconstruct: false
env:
  skip_words:
  - go
  - to
  - the
  - a
  - '[SEP]'
  name: Lorl

In [12]:
# from typing import Dict, Iterable, Callable
# import torch.nn as nn
# from torch import Tensor

# class Attention(nn.Module):
#     def __init__(self, model: nn.Module):
#         super().__init__()
#         self.model = model
#         self._attention = None

#         model.option_selector.option_dt.register_forward_hook(self.save_attention_hook())

#     def save_attention_hook(self) -> Callable:
#         def fn(model, input, output):
#             self._attention= output[-1]
#         return fn

#     def forward(self, x: Tensor) -> Dict[str, Tensor]:
#         _ = self.model(x)
#         return self._attention

In [13]:
#att_model = Attention(model)

In [35]:
from PIL import Image
from env import LorlWrapper, BabyAIWrapper
from eval import get_action

def traj(env, train_loader, model, tokenizer, args, render=False, render_freq=1, instr=None, orig_instr=None, **kwargs):
    
    if env and 'dont_reset' not in kwargs:
        if 'BabyAI' in args.env.name:
            env = BabyAIWrapper(env, train_loader.dataset)
        
        elif 'Lorl' in args.env.name:
            env = LorlWrapper(env, train_loader.dataset, instr=instr, orig_instr=orig_instr)
    
    if 'BabyAI' in args.env.name:
        max_ep_len = 300  # 2 * train_loader.dataset.max_length

        
    elif 'Lorl' in args.env.name:
        max_ep_len = 60#60 # args.env.eval_episode_factor * train_loader.dataset.max_length - 1 
            
    model.eval()

    if hasattr(model, 'module'):
        model = model.module
    else:
        model = model

    device = args.trainer.device
    method = model.method
    K = args.model.K  # 20
    horizon = model.horizon  # 20
    option_dim = model.option_dim
    model = model.to(device=device)
        
    if env:
        if method != 'vanilla':
            option_dim = model.option_selector.option_dim

        returns, lengths, successes = [], [], []
        if 'dont_reset' in kwargs:
            observation = kwargs['dont_reset']
        else:
            observation = env.reset()
        state, lang = observation['state'], observation['lang']

        state_dim = env.state_dim
        act_dim = env.act_dim
        cur_state = torch.from_numpy(state)

        # we keep all the histories on the device
        # note that the latest action and reward will be "padding"
        lm_input = tokenizer(text=[lang], add_special_tokens=True,
                                  return_tensors='pt', padding=True).to(device=device)
        with torch.no_grad():
            lm_embeddings = model.lm(
                lm_input['input_ids'], lm_input['attention_mask']).last_hidden_state
            cls_embeddings = lm_embeddings[:, 0, :]
            word_embeddings = lm_embeddings[:, 1:, :]      # skip the CLS and SEP tokens. here there's no padding so this is actually the CLS and SEP
            # word_embeddings = lm_embeddings
        
        if isinstance(state_dim, tuple):
            states = torch.from_numpy(state).reshape(1, *state_dim).to(device=device, dtype=torch.float32)
        else:
            states = torch.from_numpy(state).reshape(1, state_dim).to(device=device, dtype=torch.float32)
        actions = torch.zeros((0, act_dim), device=device, dtype=torch.float32)
        timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)
        if method != 'vanilla':
            options = torch.zeros((0, option_dim), device=device, dtype=torch.float32)

        episode_return, episode_length, success = 0, 0, 0
        options_list = []
        images = []

        option = None

        for t in range(max_ep_len):
            # add dummy action
            actions = torch.cat([actions, torch.zeros((1, act_dim), device=device)], dim=0)
            if method != 'vanilla':
                options = torch.cat([options, torch.zeros((1, option_dim), device=device)], dim=0)
            else:
                options = None

            action, option, states, actions, timesteps, options = get_action(
                model, states, actions, options, timesteps, cls_embeddings, word_embeddings, options_list, cur_state,
                option, t, horizon, K, method, state_dim, act_dim, option_dim, device, **kwargs)

            if model.decision_transformer.discrete:
                actions[-1] = torch.nn.functional.one_hot(action, num_classes=act_dim)
            else:
                action = torch.clamp(action, torch.from_numpy(env.action_space.low).to(
                    device), torch.from_numpy(env.action_space.high).to(device))
                actions[-1] = action

            action = action.detach().cpu().numpy()
            assert action in env.action_space, "Transformer predicted action outside env action space"

            obs, reward, done, info = env.step(action)
            if render:
                images.append(env.get_image())

            state, lang = obs['state'], obs['lang']
            if isinstance(state_dim, tuple):
                cur_state = torch.from_numpy(state).to(device=device).reshape(1, *state_dim)
            else:
                cur_state = torch.from_numpy(state).to(device=device).reshape(1, state_dim)
            states = torch.cat([states, cur_state], dim=0).float()
            timesteps = torch.cat(
                [timesteps,
                    torch.ones((1, 1), device=device, dtype=torch.long) * (t + 1)], dim=1)

            episode_return += reward
            episode_length += 1

            if done:
                success = info['success']
                break
            
        if render and 'dont_reset' not in kwargs:
            imageio.mimsave(f'gifs/{instr}.gif', images)

    return  word_embeddings, states.to(dtype=torch.float32), timesteps.to(dtype=torch.long), images, lm_input, options_list, episode_length, success, (obs, env)

In [15]:
import random
from utils import LORL_EVAL_INSTRS, LORL_COMPOSITION_INSTRS
import copy

# kwargs = {}
# if 'Lorl' in args.env.name:
#     orig_instr, rephrasals = random.choice(list(LORL_EVAL_INSTRS.items()))
#     rephrasal_type, instr_list = random.choice(list(rephrasals.items()))
#     instr = random.choice(instr_list)
#     kwargs['orig_instr'] = orig_instr
#     kwargs['instr'] = instr
                        
# words, states, timesteps, images, lm_inputs, options_list, episode_length, success, _ = traj(copy.deepcopy(env), train_loader, model, tokenizer, args, **kwargs)
# print(f'Generated episode of length: {episode_length}')
# print(f'Generated episode success: {success}')

In [16]:
# att_model.model.option_selector.get_option(words, states, timesteps)

In [17]:
#!pip install bertviz wordcloud

In [18]:
# from bertviz import head_view, model_view
# from transformers import BertTokenizer, BertModel

# def get_tokens(inputs):
#     # model_version = 'bert-base-uncased'
#     # do_lower_case = True
#     # model = BertModel.from_pretrained(model_version, output_attentions=True)
#     # tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)
#     # sentence_a = "The cat sat on the mat"
#     # sentence_b = "The cat lay on the rug"
#     # inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)

#     input_ids = inputs['input_ids']
#     # token_type_ids = inputs['token_type_ids']
#     # attention = model(input_ids, token_type_ids=token_type_ids)[-1]
#     # sentence_b_start = token_type_ids[0].tolist().index(1)
#     input_id_list = input_ids[0].tolist() # Batch index 0
#     tokens = tokenizer.convert_ids_to_tokens(input_id_list)[1:] 
#     return tokens

In [19]:
# tokens = get_tokens(lm_inputs)
# N = len(tokens)

In [20]:
# attention = att_model._attention
# L = min(episode_length, max_length)
# P = max_length -  L
# idx = [True] * N + [False] * P + [True] * L
# print(N, P, L)
# out = []

# for layer in range(len(attention)):
#     x = attention[layer][:, :, idx, :]
#     x = x[:, :, :, idx]
#     out.append(x)
# print(len(out))
# print(out[-1].shape)

In [21]:
# Load model and retrieve attention weights
# options = np.repeat(options_list, args.model.horizon)[:L]

# out[-1].shape

In [22]:
# options_list

In [23]:
#head_view(out, tokens + [f's_{i}, o_{o}' for i, o in enumerate(options)], layer=5)
# head_view(out, tokens + [f's_{i}, o_{o}' for i, o in enumerate(options)], layer=args.option_selector.option_transformer.n_layer-1)

In [24]:
# import matplotlib.pyplot as plt

# for i, im in enumerate(images):
#     print(f'state: {i}')
#     plt.figure(figsize = (8,8))
#     plt.imshow(im)
#     plt.show()

In [25]:
# Visualize word bags

In [26]:
# from tqdm import tqdm

# num_options = args.option_selector.num_options if args.method != 'vanilla' else 0

# words_dict = {i:[] for i in range(num_options)} # Create a list for each option
# num_eps = 10

# successes = []
# instr_wise_stats = {k: [] for k in LORL_EVAL_INSTRS.keys()}
# rephrasal_wise_stats = {k: [] for k in ['seen', 'unseen verb', 'unseen noun', 'unseen verb noun', 'human']}
    
# for i in range(num_eps):
#     for orig_instr, rephrasals in tqdm(LORL_EVAL_INSTRS.items()):
#         for rephrasal_type, instr_list in rephrasals.items():
#             for instr in instr_list:
#                 words, _, _, _, _, options_list, episode_length, success, _ = traj(env, train_loader, model, tokenizer, args, render=False, orig_instr=orig_instr, instr=instr)
#                 tokens = instr.split()
#                 successes.append(success)
#                 if rephrasal_type == 'seen':
#                     instr_wise_stats[orig_instr].append(success)
#                 rephrasal_wise_stats[rephrasal_type].append(success)
#                 for o in options_list:
#                     for w in tokens:
#                         words_dict[o].append(w)
# print(np.mean(successes))
# instr_wise_stats = {k: np.mean(instr_wise_stats[k]) for k in instr_wise_stats.keys()}
# rephrasal_wise_stats = {k: np.mean(rephrasal_wise_stats[k]) for k in rephrasal_wise_stats.keys()}
# print(instr_wise_stats)
# print(rephrasal_wise_stats)

100%|██████████| 6/6 [03:05<00:00, 30.86s/it]

0.23506493506493506
{'close drawer': 0.1, 'open drawer': 0.6, 'turn faucet left': 0.0, 'turn faucet right': 0.0, 'move black mug right': 0.2, 'move white mug down': 0.0}
{'seen': 0.15, 'unseen verb': 0.13333333333333333, 'unseen noun': 0.2833333333333333, 'unseen verb noun': 0.06666666666666667, 'human': 0.269811320754717}


In [37]:
from tqdm import tqdm

num_options = args.option_selector.num_options if args.method != 'vanilla' else 0

words_dict = {i:[] for i in range(num_options)} # Create a list for each option
num_eps = 10

successes = []
instr_wise_stats = {k: [] for k in LORL_COMPOSITION_INSTRS}
    
for i in range(num_eps):
    for orig_instr in tqdm(LORL_COMPOSITION_INSTRS):
        instr = orig_instr
        words, _, _, _, _, options_list, episode_length, success, _ = traj(env, train_loader, model, tokenizer, args, render=True, orig_instr=orig_instr, instr=instr)
        tokens = instr.split()
        successes.append(success)
        instr_wise_stats[orig_instr].append(success)
        for o in options_list:
            for w in tokens:
                words_dict[o].append(w)
print(np.mean(successes))
instr_wise_stats = {k: np.mean(instr_wise_stats[k]) for k in instr_wise_stats.keys()}
print(instr_wise_stats)

100%|██████████| 12/12 [02:25<00:00, 12.09s/it]

0.11666666666666667
{'open drawer and move black mug right': 0.0, 'pull the handle and move black mug down': 0.0, 'move white mug right': 0.1, 'move black mug down': 0.5, 'close drawer and turn faucet right': 0.0, 'close drawer and turn faucet left': 0.1, 'turn faucet left and move white mug down': 0.2, 'turn faucet right and close drawer': 0.0, 'move white mug down and turn faucet left': 0.2, 'close the drawer, turn the faucet left and move black mug right': 0.3, 'open drawer and turn faucet counterclockwise': 0.0, 'slide the drawer closed and then shift white mug down': 0.0}


In [28]:
# from tqdm import tqdm

# num_options = args.option_selector.num_options

# words_dict = {i:[] for i in range(num_options)} # Create a list for each option
# num_eps = 3
    
# for n in range(num_eps):
#     for i in tqdm(range(num_options)):
#         words, _, _, _, _, options_list, episode_length, success, _ = traj(env, train_loader, model, tokenizer, args, render=True, orig_instr=f'{i}_{n+1}', instr=f'{i}_{n+1}', constant_option=i)
#         #tokens = get_tokens(lm_inputs)
#         #tokens = instr.split()
#         #if success:
#         #for o in options_list:
#         #    for w in tokens:
#         #        words_dict[o].append(w)

In [29]:
# task = 'close drawer and turn faucet right'
# #options = [17, 2] 
# options = [14, 2]

# #task = 'pull the handle and move black mug down'
# #options = [9, 15]

# for n in range(3):
#     all_images = []
#     all_options = []
#     all_success = []
#     words, _, _, images, _, options_list, episode_length, success, (prev_obs, prev_env) = traj(copy.deepcopy(env), train_loader, model, tokenizer, args, render=True, orig_instr=task, instr=task, constant_option=options[0])
#     all_images.extend(images)
#     all_options.extend(options_list)
#     all_success.append(success)
#     for option in options[1:]:
#         words, _, _, images, _, options_list, episode_length, success, (prev_obs, prev_env) = traj(prev_env, train_loader, model, tokenizer, args, render=True, orig_instr=task, instr=task, constant_option=option, dont_reset=prev_obs)
#         all_images.extend(images)
#         all_options.extend(options_list)
#         all_success.append(success)
#     imageio.mimsave(f'gifs/{task}_{n+1}.gif', all_images)
#     print(all_options, sum(all_success))

In [30]:
from itertools import chain

skip_words = ['the', 'a', '[SEP]']

words = sorted(set(chain(*words_dict.values())) - set(skip_words))
print(words)

[]


In [31]:
def w_to_ind(word):
    return words.index(word)

In [32]:
matrix = np.zeros([len(words), num_options])

for o in range(num_options):
    for w in words_dict[o]:
        if w not in skip_words:
            matrix[w_to_ind(w), o] += 1
            
#print(matrix)

In [33]:
#!pip install seaborn

In [34]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize = (30,10))
sns.heatmap(matrix, yticklabels=words)
plt.plot()

ValueError: zero-size array to reduction operation fmin which has no identity

<Figure size 2160x720 with 0 Axes>

In [ ]:
matrix.sum(axis=0, keepdims=True).shape

In [ ]:
%matplotlib inline
# Now if we normalize it by column:
plt.figure(figsize = (30,10))
matrix_norm_col=(matrix)/(matrix.sum(axis=0, keepdims=True) + 1e-6)
im = sns.heatmap(matrix_norm_col, yticklabels=words)
plt.show()

In [ ]:
%matplotlib inline
# Now if we normalize it by row:
plt.figure(figsize = (30,10))
matrix_norm_row=(matrix)/(matrix.sum(axis=1, keepdims=True) + 1e-6)
sns.heatmap(matrix_norm_row, yticklabels=words)
plt.show()

In [ ]:
%matplotlib inline
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud

for i in range(num_options):
    if words_dict[i]:
        print(i)
        cloud = WordCloud(max_font_size=80,colormap="hsv").generate_from_frequencies(Counter(x for x in words_dict[i] if x not in skip_words))
        plt.figure(figsize=(16,12))
        plt.imshow(cloud, interpolation='bilinear')
        plt.axis('off')
        plt.show()

In [ ]:
#!pip install wordcloud